In [1]:
import torch

In [ ]:
import transformers
from unsloth import FastLanguageModel
from transformers import pipeline

In [2]:
model_id = "meta-llama/Llama-3.2-1B"

# Load the model in a 4-bit quantized mode
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=2048,
    load_in_4bit=True,
    dtype=torch.bfloat16
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

In [ ]:
pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

In [ ]:
output = pipe("What is the meaning of life?", max_length=50, num_return_sequences=1)
print(output)

In [3]:
print(torch.__version__)
my_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device="cpu")
print(my_tensor)
torch.cuda.is_available()

2.4.1+cu118
tensor([[1., 2., 3.],
        [4., 5., 6.]])


True